# Libraries

In [203]:
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
#from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_extraction.text import HashingVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve, auc
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from scipy.sparse import hstack, csr_matrix, vstack
from scipy.stats import spearmanr
import lightgbm as lgb
import gc
import re
from multiprocessing import Pool
import multiprocessing as mp
import psutil
import numpy as np
import pandas as pd
import random
from textblob import TextBlob
import itertools

# Data

In [2]:
train = pd.read_csv('input/train.csv')

In [3]:
train.head()

,qa_id,question_title,question_body,question_user_name,question_user_page,answer,answer_user_name,answer_user_page,url,category,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,0,What am I losing when using extension tubes in...,After playing around with macro photography on...,ysap,https://photo.stackexchange.com/users/1024,"I just got extension tubes, so here's the skin...",rfusca,https://photo.stackexchange.com/users/1917,http://photo.stackexchange.com/questions/9169/...,LIFE_ARTS,...,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.0,0.000000,0.000000,1.000000
1,1,What is the distinction between a city and a s...,I am trying to understand what kinds of places...,russellpierce,https://rpg.stackexchange.com/users/8774,It might be helpful to look into the definitio...,Erik Schmidt,https://rpg.stackexchange.com/users/1871,http://rpg.stackexchange.com/questions/47820/w...,CULTURE,...,0.888889,0.888889,0.555556,0.888889,0.888889,0.666667,0.0,0.000000,0.666667,0.888889
2,2,Maximum protusion length for through-hole comp...,I'm working on a PCB that has through-hole com...,Joe Baker,https://electronics.stackexchange.com/users/10157,Do you even need grooves? We make several pro...,Dwayne Reid,https://electronics.stackexchange.com/users/64754,http://electronics.stackexchange.com/questions...,SCIENCE,...,0.777778,0.777778,0.555556,1.000000,1.000000,0.666667,0.0,0.333333,1.000000,0.888889
3,3,Can an affidavit be used in Beit Din?,"An affidavit, from what i understand, is basic...",Scimonster,https://judaism.stackexchange.com/users/5151,"Sending an ""affidavit"" it is a dispute between...",Y e z,https://judaism.stackexchange.com/users/4794,http://judaism.stackexchange.com/questions/551...,CULTURE,...,0.888889,0.833333,0.333333,0.833333,1.000000,0.800000,0.0,0.000000,1.000000,1.000000
4,5,How do you make a binary image in Photoshop?,I am trying to make a binary image. I want mor...,leigero,https://graphicdesign.stackexchange.com/users/...,Check out Image Trace in Adobe Illustrator. \n...,q2ra,https://graphicdesign.stackexchange.com/users/...,http://graphicdesign.stackexchange.com/questio...,LIFE_ARTS,...,1.000000,1.000000,0.666667,1.000000,1.000000,0.800000,1.0,0.000000,1.000000,1.000000


In [5]:
sub = pd.read_csv('input/sample_submission.csv')

In [6]:
sub.head()

,qa_id,question_asker_intent_understanding,question_body_critical,question_conversational,question_expect_short_answer,question_fact_seeking,question_has_commonly_accepted_answer,question_interestingness_others,question_interestingness_self,question_multi_intent,...,question_well_written,answer_helpful,answer_level_of_information,answer_plausible,answer_relevance,answer_satisfaction,answer_type_instructions,answer_type_procedure,answer_type_reason_explanation,answer_well_written
0,39,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,...,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308,0.00308
1,46,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,...,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448,0.00448
2,70,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,...,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673,0.00673
3,132,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,...,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401,0.01401
4,200,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,...,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074,0.02074


In [7]:
# First row
train.iloc[0,:]

qa_id                                                                                    0
question_title                           What am I losing when using extension tubes in...
question_body                            After playing around with macro photography on...
question_user_name                                                                    ysap
question_user_page                              https://photo.stackexchange.com/users/1024
answer                                   I just got extension tubes, so here's the skin...
answer_user_name                                                                    rfusca
answer_user_page                                https://photo.stackexchange.com/users/1917
url                                      http://photo.stackexchange.com/questions/9169/...
category                                                                         LIFE_ARTS
host                                                               photo.stackexchange.com

In [15]:
dict(train.category.value_counts())

{'TECHNOLOGY': 2441,
 'STACKOVERFLOW': 1253,
 'CULTURE': 963,
 'SCIENCE': 713,
 'LIFE_ARTS': 709}

In [16]:
category_freq_encoding_dict = {
    'TECHNOLOGY': 0,
    'STACKOVERFLOW': 1,
    'CULTURE': 2,
    'SCIENCE': 3,
    'LIFE_ARTS': 4
}

In [17]:
text_columns = ['question_title', 'question_body', 'answer']

In [18]:
target_columns = ['question_asker_intent_understanding',
'question_body_critical',
'question_conversational',
'question_expect_short_answer',
'question_fact_seeking',
'question_has_commonly_accepted_answer',
'question_interestingness_others',
'question_interestingness_self',
'question_multi_intent',
'question_not_really_a_question',
'question_opinion_seeking',
'question_type_choice',
'question_type_compare',
'question_type_consequence',
'question_type_definition',
'question_type_entity',
'question_type_instructions',
'question_type_procedure',
'question_type_reason_explanation',
'question_type_spelling',
'question_well_written',
'answer_helpful',
'answer_level_of_information',
'answer_plausible',
'answer_relevance',
'answer_satisfaction',
'answer_type_instructions',
'answer_type_procedure',
'answer_type_reason_explanation',
'answer_well_written']

In [19]:
len(target_columns)

30

In [20]:
for i in target_columns:
    print(train[i].value_counts())

1.000000    2833
0.888889    1579
0.777778     883
0.666667     503
0.833333     108
0.333333      68
0.555556      60
0.500000      33
0.444444      12
Name: question_asker_intent_understanding, dtype: int64
0.333333    1506
0.555556     932
0.444444     893
0.666667     845
0.777778     701
1.000000     583
0.888889     470
0.500000      94
0.833333      55
Name: question_body_critical, dtype: int64
0.000000    5409
0.333333     384
0.666667     152
1.000000     104
0.500000      30
Name: question_conversational, dtype: int64
1.000000    2936
0.666667    1373
0.000000     806
0.333333     522
0.500000     442
Name: question_expect_short_answer, dtype: int64
1.000000    3480
0.666667    1326
0.333333     877
0.000000     315
0.500000      81
Name: question_fact_seeking, dtype: int64
1.000000    4071
0.666667     794
0.000000     657
0.333333     324
0.500000     233
Name: question_has_commonly_accepted_answer, dtype: int64
0.666667    1831
0.555556    1699
0.444444    1258
0.777778   

In [221]:
CHARS_TO_REMOVE = '!¡"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n“”’\'∞θ÷α•à−β∅³π‘₹´°£€\×™√²—'
MAX_LEN = 100000
STOP_WORDS = list(stopwords.words('english'))
num_partitions = psutil.cpu_count() * 8
num_cores = psutil.cpu_count()

In [222]:
vectorizer = TfidfVectorizer(ngram_range=(1,2),
                             min_df=3, max_df=0.3,
                             strip_accents='unicode',
                             use_idf=1,
                             smooth_idf=1,
                             sublinear_tf=1,
                             max_features=MAX_LEN,
                             lowercase=False)

In [223]:
def remove_stop_words(text, sw):
    text = ' '.join([word for word in text.split() if word not in sw])
    return text

def remove_noise_chars(text, chars):
    text = ''.join([word for word in text if word not in chars])
    return text

def c_t2(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r"\'scuse", " excuse ", text)
    text = re.sub('\W', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = re.sub(r"\\", "", text)    
    text = re.sub(r"\'", "", text)    
    text = re.sub(r"\"", "", text)
    text = re.sub(r'<.*?>', '', text)
    text = text.strip(' ')
    return text


In [224]:
def vector(text_data, train=True, to_dataframe=False):

    if train:
        vectorizer.fit(text_data)
        
    X_words = vectorizer.transform(text_data)
    
    if to_dataframe:
        X_words = pd.DataFrame(X_words)            

    return X_words

In [245]:
def full_cleaner(df, col):
    df['{}_cleaned'.format(col)] = df[col].apply(lambda x: c_t2(x))
#     df['{}_cleaned'.format(col)] = df['{}_cleaned'.format(col)].apply(lambda x: remove_stop_words(x, STOP_WORDS))
    df['{}_cleaned'.format(col)] = df['{}_cleaned'.format(col)].apply(lambda x: remove_noise_chars(x, CHARS_TO_REMOVE))
    df['{}_cleaned'.format(col)] = df['{}_cleaned'.format(col)].apply(lambda x: ''.join([i for i in x if not i.isdigit()]))
#     df['{}_cleaned'.format(col)] = df['{}_cleaned'.format(col)].apply(lambda x: ''.join(''.join(s)[:1] for _, s in itertools.groupby(x)))
    return df

In [246]:
train = full_cleaner(train, 'question_title')
train['question_title_cleaned'].head()

0    what am i losing when using extension tubes in...
1    what is the distinction between a city and a s...
2    maximum protusion length for through hole comp...
3                 can an affidavit be used in beit din
4          how do you make a binary image in photoshop
Name: question_title_cleaned, dtype: object

In [247]:
train_qtc = vector(train['question_title_cleaned'])

In [248]:
train = full_cleaner(train, 'question_body')
train['question_body_cleaned'].head()

0    after playing around with macro photography on...
1    i am trying to understand what kinds of places...
2    i am working on a pcb that has through hole co...
3    an affidavit from what i understand is basical...
4    i am trying to make a binary image i want more...
Name: question_body_cleaned, dtype: object

In [249]:
train_qbc = vector(train['question_body_cleaned'])

In [250]:
X = hstack([train_qtc, train_qbc])

In [251]:
# X = hstack([vector(train['question_title']),
#             vector(train['question_body'])])

In [252]:
X.shape

(6079, 71084)

Custom metric for LightGBM

In [290]:
def custom_metric(y_pred, y_true):
    corr = spearmanr(y_true, y_pred).correlation
    return 'Spearman\'s correlation coefficient', corr, True

# Multiclass classifier

In [297]:
verbose = 50
EARLY_STOPPING_ROUNDS = 100
FT_SEL = False
k = 3
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
params = {
        'max_depth': 8,
#         'metric': 'multi_logloss',
        'n_estimators': 20000,
        'learning_rate': 0.1,
       'num_leaves': 56,
#        'min_data_in_leaf': 20,
        'colsample_bytree': 0.9,
#        "is_unbalance": True,
        'objective': 'multi_error',
#         'scale_pos_weight': 10,
        'n_jobs': -1,
        'seed': 42,
        'bagging_fraction': 0.5,
        'lambda_l1': 0,
        'lambda_l2': 0,
    }
lgb_model = lgb.LGBMClassifier(**params)

In [298]:
def fit_train(X, target, esr, verbose):

    le = LabelEncoder()

    first_target = le.fit_transform(train[target])

    X_train, X_test, y_train, y_test = train_test_split(X, first_target,
                                                        test_size=0.2,
                                                        random_state=42)
    
    lgb_model.fit(X_train,
                  y_train,
                  eval_set=[(X_test, y_test)],
                  verbose=verbose,
                  early_stopping_rounds=esr,
#                   eval_metric=custom_metric
                 )

    solution = pd.DataFrame({'y_true': le.inverse_transform(y_test),
                             'y_pred': le.inverse_transform(lgb_model.predict(X_test))})

    score = spearmanr(solution.y_true, solution.y_pred).correlation
    
    return solution, score

In [300]:
scores = list()
for y in target_columns[:10]:
    print(f'Training with target: {y}')
    solution, score = fit_train(X, y, EARLY_STOPPING_ROUNDS, verbose)
    scores.append(score)
    print(f'\tScore: {score}')
    print(f'\tCurrent mean score: {np.nanmean(scores)}\n')
    counter += 1

Training with target: question_asker_intent_understanding
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_logloss: 1.39973
[100]	valid_0's multi_logloss: 1.44688
Early stopping, best iteration is:
[18]	valid_0's multi_logloss: 1.38282
	Score: 0.08647420492930341
	Current mean score: 0.08647420492930341

Training with target: question_body_critical
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_logloss: 1.88373
[100]	valid_0's multi_logloss: 1.92734
Early stopping, best iteration is:
[32]	valid_0's multi_logloss: 1.87702
	Score: 0.46510099916901393
	Current mean score: 0.27578760204915864

Training with target: question_conversational
Training until validation scores don't improve for 100 rounds.
[50]	valid_0's multi_logloss: 0.404259
[100]	valid_0's multi_logloss: 0.431895
Early stopping, best iteration is:
[30]	valid_0's multi_logloss: 0.399421
	Score: 0.21688348594209056
	Current mean score: 0.2561528966801359



In [220]:
np.unique(le.inverse_transform(lgb_model.predict(X_test)))

array([0.33333333, 0.44444444, 0.5       , 0.55555556, 0.66666667,
       0.77777778, 0.83333333, 0.88888889, 1.        ])

# Binary classifier

In [193]:
first_target = np.where(train[target_columns[1]].fillna(0) >= 0.5, 1, 0)

In [194]:
first_target[:4]

array([0, 1, 1, 1])

In [195]:
X_train, X_test, y_train, y_test = train_test_split(X, first_target,
                                                    test_size=0.2,
                                                    random_state=42)

In [196]:
verbose = 50
EARLY_STOPPING_ROUNDS = 100
FT_SEL = False
k = 3
skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
params = {
        'max_depth': 5,
#         'metric': 'auc',
        'n_estimators': 20000,
        'learning_rate': 0.05,
#        'num_leaves': 30,first_target
#        'min_data_in_leaf': 20,
        'colsample_bytree': 0.7,
#        "is_unbalance": True,
        'objective': 'xentropy',
#         'scale_pos_weight': 10,
        'n_jobs': -1,
        'seed': 42,
        'bagging_fraction': 0.5,
        'lambda_l1': 0,
        'lambda_l2': 0,
    }
lgb_model = lgb.LGBMClassifier(**params)

In [197]:
lgb_model.fit(X_train,
              y_train,
              eval_set=[(X_test, y_test)],
              verbose=verbose,
              early_stopping_rounds=EARLY_STOPPING_ROUNDS,
        #   eval_metric=custom_loss
                  )

Training until validation scores don't improve for 100 rounds.
[50]	valid_0's xentropy: 0.56292
[100]	valid_0's xentropy: 0.545919
[150]	valid_0's xentropy: 0.539365
[200]	valid_0's xentropy: 0.537143
[250]	valid_0's xentropy: 0.535289
[300]	valid_0's xentropy: 0.534592
[350]	valid_0's xentropy: 0.53481
[400]	valid_0's xentropy: 0.5347
[450]	valid_0's xentropy: 0.537533
Early stopping, best iteration is:
[356]	valid_0's xentropy: 0.534296


LGBMClassifier(bagging_fraction=0.5, boosting_type='gbdt', class_weight=None,
               colsample_bytree=0.7, importance_type='split', lambda_l1=0,
               lambda_l2=0, learning_rate=0.05, max_depth=5,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=20000, n_jobs=-1, num_leaves=31,
               objective='xentropy', random_state=None, reg_alpha=0.0,
               reg_lambda=0.0, seed=42, silent=True, subsample=1.0,
               subsample_for_bin=200000, subsample_freq=0)

In [198]:
solution_0 = pd.DataFrame({'y_true': y_test,
                           'y_pred': lgb_model.predict_proba(X_test)[:,1]})

In [199]:
solution_0

,y_true,y_pred
0,0,0.508424
1,1,0.637387
2,1,0.952894
3,1,0.499025
4,1,0.785161
...,...,...
1211,1,0.396578
1212,1,0.786138
1213,1,0.451353
1214,1,0.841534


In [200]:
solution_0.head()

,y_true,y_pred
0,0,0.508424
1,1,0.637387
2,1,0.952894
3,1,0.499025
4,1,0.785161


In [201]:
solution_0[solution_0.y_true == 0].head()

,y_true,y_pred
0,0,0.508424
6,0,0.421058
7,0,0.669666
8,0,0.298424
9,0,0.722702


In [99]:
solution_0[solution_0.y_pred == 0]

,y_true,y_pred
